In [26]:
from py2neo import Graph
from py2neo import Node, Relationship
import pandas as pd
import numpy as np

# For legacy INDEXER
from py2neo import neo4j

g = Graph()
g.delete_all()

idx = g.legacy.get_or_create_index(neo4j.Node, "Vertex")
print(idx)

Index(Node, 'http://localhost:7474/db/data/index/node/Vertex')


In [15]:
df = pd.read_csv('./AtgO2.txt', sep="\t", names=["target", "source", "type"])

nodes = []

s = df["source"].astype(str)
t = df["target"].astype(str)

all_nodes = s.append(t).unique()
print(len(all_nodes))

712


In [16]:
# Assign genes
df2 = pd.read_csv('AtgO2_terms_to_genes.txt', sep="\t", names=["term", "genes"])
df2.head()

,term,genes
0,1,RIM13|VPS4|IWR1|VPS21|MSN1|SSD1|AKR1|AKR2|GCN2...
1,2,RIM13|VPS4|IWR1|VPS21|MSN1|SSD1|AKR1|AKR2|GCN2...
2,3,RIM13|VPS4|IWR1|VPS21|MSN1|SSD1|AKR1|AKR2|GCN2...
3,5,VPS4|IWR1|VPS21|SSD1|AKR1|AKR2|GCN2|GCN4|CTK1|...
4,6,VPS4|IWR1|VPS21|GCN2|GCN4|CTK1|SEC7|DOM34|HBS1...


In [17]:
assigned_genes = {}

for row in df2.itertuples():
    genes = row[2]
    gene_list = genes.split('|')
    assigned_genes[str(row[1])] = gene_list
    
df.head()

,target,source,type
0,212,RIM13,gene
1,212,VPS4,gene
2,174,IWR1,gene
3,174,VPS21,gene
4,172,MSN1,gene


In [27]:
nodes = {}

for row in df.itertuples():
    source_name = str(row[1])
    target_name = str(row[2])
    
    if source_name in nodes:
        source = nodes[source_name]
    else:
        if source_name in assigned_genes:
            source = idx.get_or_create(
                "name", source_name, {
                    "name": source_name,
                    "assigned_genes": assigned_genes[source_name]
                })
        else:
            source = idx.get_or_create(
                "name", source_name, {
                    "name": source_name
                })
        
        nodes[source_name] = source
    
    if target_name in nodes:
        target = nodes[target_name]
    else:
        if target_name in assigned_genes:
            target = idx.get_or_create(
                "name", target_name, {
                    "name": target_name,
                    "assigned_genes": assigned_genes[target_name]
                })
        else:
            target = idx.get_or_create(
                "name", target_name, {
                    "name": target_name
                })
        nodes[target_name] = target
       
    
    edge = Relationship(source, "child_of", target)
    g.create(edge)

In [25]:
print(g.size)

results = idx.query("name:*")
for r in results:
    print(r)

1445
(n2138 {name:"RIM13"})
(n2142 {name:"VPS21"})
(n2139 {name:"VPS4"})
(n2141 {name:"IWR1"})
(n2140 {assigned_genes:["IWR1","VPS21"],name:"174"})
(n2137 {assigned_genes:["RIM13","VPS4"],name:"212"})
(n2144 {name:"MSN1"})
(n2145 {name:"SSD1"})
(n2147 {name:"AKR1"})
(n2148 {name:"AKR2"})
(n2150 {name:"GCN2"})
(n2151 {name:"GCN4"})
(n2143 {assigned_genes:["MSN1","SSD1"],name:"172"})
(n2146 {assigned_genes:["AKR1","AKR2"],name:"180"})
(n2149 {assigned_genes:["GCN2","GCN4"],name:"179"})
(n2153 {name:"CTK1"})
(n2154 {name:"SGV1"})
(n2156 {name:"PTK2"})
(n2157 {name:"PTK1"})
(n2159 {name:"ARF1"})
(n2160 {name:"SEC7"})
(n2152 {assigned_genes:["CTK1","SGV1"],name:"177"})
(n2155 {assigned_genes:["PTK2","PTK1"],name:"176"})
(n2158 {assigned_genes:["ARF1","SEC7"],name:"175"})
(n2163 {name:"YDL199C"})
(n2162 {name:"VPS73"})
(n2165 {name:"DOM34"})
(n2166 {name:"HBS1"})
(n2168 {name:"ATG8"})
(n2169 {name:"SNX4"})
(n2164 {assigned_genes:["DOM34","HBS1"],name:"173"})
(n2161 {assigned_genes:["VPS73","

In [ ]:
for row in df2.itertuples():
    genes = row.genes
    gene_list = genes.split('|')
    print(len(gene_list))

In [ ]:
for line in raw_interactions:
  new_line = line.rstrip()

  vals = new_line.split("\t")
  source = g.get_indexed_node("Vertex", "name", vals[0])
  target = g.get_indexed_node("Vertex", "name", vals[2])
  if source is not None and target is not None:
    newEdge = g.create(rel(source,"raw_interaction",target))

raw_interactions.close()

print(g.size())